In [21]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt

In [22]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visitsItems = pd.read_sql('SELECT * FROM VisitsItems', con=db_connection)
visitsItems['created'] = pd.to_datetime(visitsItems['created'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visitsItems['lastmodified'] = pd.to_datetime(visitsItems['lastmodified'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visitsItems['deleted'].replace(0, np.nan, inplace=True)
visitsItems['deleted'] = pd.to_datetime(visitsItems['deleted'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visitsItems['title'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
visitsItems['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
visitsItems['title'] = visitsItems['title'].astype('category')
visitsItems['description'] = visitsItems['description'].astype('category')

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", 1000)

In [23]:
visitsItems.set_index('created', inplace=True)
visitsItem_openinghours = visitsItems.between_time('10:00', '18:00')
visitsItems.reset_index(inplace=True)
visitsItem_openinghours.reset_index(inplace=True)

# collected per year

In [5]:
visitsItems_collected = visitsItem_openinghours[visitsItem_openinghours['action_id'] == 1]
visitsItems_collected['created'].groupby(visitsItems_collected.created.dt.to_period("Y")).agg('count')

/Users/shirin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1102: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


created
2014        972
2015    2848435
2016    7588938
2017    3344537
2018    1581586
2019    1304527
2020     241677
Freq: A-DEC, Name: created, dtype: int64

## created per year

In [7]:
visitsItems_created = visitsItem_openinghours[visitsItem_openinghours['action_id'] == 2]
visitsItems_created['created'].groupby(visitsItems_created.created.dt.to_period("Y")).agg('count')

created
2014        37
2015     96250
2016    129156
2017     75204
2018     29831
2019     25738
2020      3356
Freq: A-DEC, Name: created, dtype: int64

## average items per user w/o user==0

In [8]:
visitsItems_not_zero = visitsItem_openinghours[visitsItem_openinghours['user_id'] != 0]
visitsItems_not_zero_and_action1 = visitsItems_not_zero[visitsItems_not_zero['action_id'] == 1]
visitsItems_action1 = visitsItems[visitsItems['action_id'] == 1]


In [11]:
group_visit_items= visitsItems_not_zero_and_action1.groupby(['user_id']).agg('sum')
group_visit_items.reset_index(inplace=True)
group_visit_count = group_visit_items[['user_id', 'action_id']]
group_visit_count.sort_values(by="action_id", ascending=False, inplace=True)
group_visit_count['action_id'].mean()

/Users/shirin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


69.01548492257538

## collected vs created

In [24]:
visitsItem_openinghours = visitsItem_openinghours[visitsItem_openinghours['created'] > '2010']
visitsItem_openinghours['created'] = visitsItem_openinghours['created'].dt.to_period("Y")
grouped_year_action = visitsItem_openinghours.groupby(['action_id', 'created']).agg({'created': ['count']})
grouped_year_action.rename({'created': 'count_of_action'}, axis=1, inplace=True)
grouped_year_action = grouped_year_action.unstack(level=0)
grouped_year_action

/Users/shirin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1102: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


count_of_action        
                    count        
action_id               1       2
created                          
2014                  972      37
2015              2848435   96250
2016              7588938  129156
2017              3344537   75204
2018              1581586   29831
2019              1304527   25738
2020               241677    3356